In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install pandas
!pip install sqlite3
!pip install transformers
!pip install torch

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.pipeline import make_pipeline
# import joblib
# import sqlite3

# # Load training data
# df = pd.read_csv('./intents_and_examples.csv')

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df['Example'], df['Intent'], test_size=0.2, random_state=42)

# # Create a pipeline that vectorizes the text data and then applies a classifier
# model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# # Train the model
# model.fit(X_train, y_train)

# # Save the model
# joblib.dump(model, './intent_classifier.pkl')

# # Evaluate the model
# accuracy = model.score(X_test, y_test)
# print(f"Model accuracy: {accuracy:.2f}")

In [ ]:
import pandas as pd
import sqlite3

# Load the CSV file
financial_data = pd.read_csv('./Dummy_Financial_Data.csv')

# Connect to SQLite database (or create it)
conn = sqlite3.connect('./financial_data.db')

# Convert the DataFrame to a SQL table
financial_data.to_sql('financial_data', conn, if_exists='replace', index=False)

# Verify the table
query = "SELECT * FROM financial_data LIMIT 5;"
financial_data_sample = pd.read_sql(query, conn)
print(financial_data_sample)

# Close the connection
conn.close()

   Account Number            Name  Balance                       Email  \
0      7230780854      John White  5960.25    casey.thomas@example.com   
1      8736393608  Casey Anderson  9245.09     cameron.white@yahoo.com   
2      2441783699        Jane Doe  6483.57       john.taylor@yahoo.com   
3      2452236112    Taylor Smith  5414.68    john.johnson@example.com   
4      2621649680    Jordan Smith  9384.00  cameron.taylor@example.com   

   Phone Number Account Type  Credit Score                       Address  
0    6916345629         Loan           592    851 Maple St, Fairview, NY  
1    3145173384      Savings           801  960 Broadway, Greenville, TX  
2    3104465091         Loan           840   854 Park Ave, Riverside, NY  
3    8774850601     Checking           329    789 Broadway, Fairview, PA  
4    7810087179       Credit           643   711 Maple St, Riverside, TX  


In [ ]:
import pandas as pd
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the intents and examples
intents_df = pd.read_csv('intents_and_examples.csv')

# Prepare the dataset
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Encode the labels
label_map = {label: idx for idx, label in enumerate(intents_df['Intent'].unique())}
intents_df['label'] = intents_df['Intent'].map(label_map)

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    intents_df['Example'].tolist(),
    intents_df['label'].tolist(),
    test_size=0.2,
    random_state=42
)

train_dataset = IntentDataset(train_texts, train_labels, tokenizer, max_len=64)
val_dataset = IntentDataset(val_texts, val_labels, tokenizer, max_len=64)

# Load pre-trained XLM-RoBERTa model
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=len(label_map))

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=30,
)

# Define compute_metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('./financial_xlm_roberta_model')
tokenizer.save_pretrained('./financial_xlm_roberta_tokenizer')

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Test Accuracy: {eval_results['eval_accuracy']:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
30,3.009600
60,3.017300
90,2.951900
120,2.903500
150,2.864600
180,2.746000
210,2.712900
240,2.736400
270,2.689900
300,2.417200


Test Accuracy: 0.7311


In [ ]:
import pandas as pd
import sqlite3
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
import torch

# Load the trained model and tokenizer
model = XLMRobertaForSequenceClassification.from_pretrained('./financial_xlm_roberta_model')
tokenizer = XLMRobertaTokenizer.from_pretrained('./financial_xlm_roberta_tokenizer')
model.eval()

# Connect to the database
def query_database(query, params=()):
    conn = sqlite3.connect('./financial_data.db')
    cursor = conn.cursor()
    cursor.execute(query, params)
    result = cursor.fetchall()
    conn.close()
    return result

# Load the intents and examples to get the label_map
intents_df = pd.read_csv('./intents_and_examples.csv')
label_map = {label: idx for idx, label in enumerate(intents_df['Intent'].unique())}

# Map intent ID to label
def get_intent_label(intent_id):
    return list(label_map.keys())[list(label_map.values()).index(intent_id)]

# Function to process user input and generate response
def process_input(user_input, account_number=None):
    if account_number is None:
        if user_input.isdigit():  # Assuming account numbers are digits
            return None, f"Account number {user_input} recognized. How can I help you with your account today?", user_input
        else:
            return None, "Please provide your account number.", None

    # Process the user's request with the account number
    inputs = tokenizer(user_input, return_tensors='pt', truncation=True, padding=True, max_length=64)
    outputs = model(**inputs)
    intent_id = torch.argmax(outputs.logits).item()
    intent = get_intent_label(intent_id)

    print(f"DEBUG: User Input: {user_input}, Detected Intent: {intent}")

    if intent == 'check_balance':
        response = query_database("SELECT balance FROM financial_data WHERE [Account Number] = ?", (account_number,))
        response_text = f"Your account balance is {response[0][0]}" if response else "Account not found."
    elif intent == 'transaction_history':
        response = query_database("SELECT * FROM financial_data WHERE account_id = ? LIMIT 5", (account_number,))
        response_text = response if response else "No transaction history found."
    elif intent == 'check_human':
        response_text = "I am an AI created to assist you with financial inquiries."
    elif intent == 'open_account':
        response_text = "To open a new account, please visit our website or nearest branch with your identification documents."
    elif intent == 'close_account':
        response_text = "To close your account, please visit our nearest branch or contact our customer support."
    elif intent == 'loan_inquiry':
        response_text = "You can inquire about loans and their eligibility criteria on our website or by visiting our branch."
    elif intent == 'credit_card_application':
        response_text = "You can apply for a credit card through our website or by visiting our nearest branch."
    elif intent == 'contact_support':
        response_text = "You can contact our customer support through the chat feature on our website or by calling our support number."
    else:
        response_text = "I'm sorry, I didn't understand that."

    return intent, response_text, account_number

# Main loop to run the chatbot in the terminal
def run_chatbot():
    account_number = None
    print("Welcome to the Financial Chatbot!")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Chatbot: Goodbye!")
            break
        intent, response, account_number = process_input(user_input, account_number)
        print(f"Chatbot: {response}")

# Run the chatbot
run_chatbot()


Welcome to the Financial Chatbot!
Chatbot: Please provide your account number.
Chatbot: Account number 2621649680 recognized. How can I help you with your account today?
DEBUG: User Input: give me balance, Detected Intent: help
Chatbot: I'm sorry, I didn't understand that.
DEBUG: User Input: give me account balance, Detected Intent: check_recipients
Chatbot: I'm sorry, I didn't understand that.
DEBUG: User Input: what is my balance, Detected Intent: check_recipients
Chatbot: I'm sorry, I didn't understand that.
DEBUG: User Input: what is my account balance, Detected Intent: check_balance
Chatbot: Your account balance is 9384.0
DEBUG: User Input: are you a human?, Detected Intent: check_human
Chatbot: I am an AI created to assist you with financial inquiries.
DEBUG: User Input: i want to close my account, Detected Intent: pay_cc
Chatbot: I'm sorry, I didn't understand that.
DEBUG: User Input: how to close my account?, Detected Intent: check_balance
Chatbot: Your account balance is 9384.